In [ ]:
import torch
import torchvision

In [ ]:
from torch import nn
class Downsample_block(nn.Module):
    def __init__(self,in_channels=3,out_channels=64,kernel_size = 3,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.layer1 = nn.Conv2d(in_channels,out_channels,kernel_size)
        self.relu1 = nn.ReLU(inplace=True)
        self.layer2 = nn.Conv2d(out_channels,out_channels,kernel_size)
        self.relu2 = nn.ReLU(inplace=True)
        self.pooling = nn.MaxPool2d(kernel_size=2,stride=2)

    def forward(self,x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        return self.pooling(x)
    
    def copy(self,x):
        with torch.no_grad():
            x = self.layer1(x)
            x = self.relu1(x)
            x = self.layer2(x)
            x = self.relu2(x)
            return x



In [ ]:
class Upsampler(nn.Module):
    def __init__(self, in_channels,mid_channels,out_channels,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.mod = nn.Sequential(
         nn.Conv2d(in_channels,mid_channels,3),
         nn.ReLU(),
         nn.Conv2d(mid_channels,mid_channels,3),
         nn.Conv2d(mid_channels,mid_channels,3),
         nn.ConvTranspose2d(mid_channels,out_channels,2))
    def forward(self,x):
        return self.mod(x)

In [ ]:
class Hnet(nn.Module):
    def __init__(self,in_channels,num_classes,*args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.downsampler1 = Downsample_block(in_channels)
        self.downsampler2 = Downsample_block(64,128)
        self.downsampler3 = Downsample_block(128,256)
        self.downsampler4 = Downsample_block(256,512)
        self.upsampler1 = Upsampler(512,1024,512)
        self.upsampler2 = Upsampler(1024,512,256)
        self.upsampler3 = Upsampler(512,256,128)
        self.upsampler4 = Upsampler(256,128,64)
        self.compute_res = nn.Sequential(
            *list(Downsample_block(128,64).children())[:-1],
            nn.Conv2d(64,num_classes,1)
        )
        self.attention = 
    
    def cut_tensor(X,target_tensor):
        shape_tgt = target_tensor.shape[-1]#图片是正方形的
        shape_src = X.shape[-1]
        delta = (shape_src-shape_tgt)/2
        return X[:,:,delta:shape_src-delta,delta:shape_src-delta]